# Data fetching and Dataframe manipulation 

# TO DO: 
- Add .env file 
- Add keggle csv
- Add Short SQL overviw 
- Tipps for SQL, name and describe libraries that can be used for SQL (psycopg2 and )

- MAKE IT RUN 

## Creating a connection to a PostgreSQL database with Python


In order to create a connection to our PostgreSQL database we need the following information:

- host = the address of the machine the database is hosted on
- port = the virtual gate number through which communication will be allowed
- database = the name of the database
- user = the name of the user
- password = the password of the user

Because we don't want that the database information is published on github we put it into a .env file which is added into the gitignore. 
In these kind of files you can store information that is not supposed to be published.
With the `dotenv` package you can read the `.env` files and get the variables.
The file was 'force added' to the repo using ```git add -f .env``` command. Please follow instructions inside the file to ensure you have the right credentials inside. 



In [ ]:
# Using psycopg2

import pandas as pd # to read sql data into a pandas dataframe
import psycopg2 # to connect to SQL database
import os # provides functions for interacting with operating system
from dotenv import load_dotenv # reads key-value pairs from a .env file and can set them as environment variables

load_dotenv() # take environment variables from .env
DATABASE = os.getenv('DATABASE')
USER_DB = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')


# Create / open connection object conn (no need to edit code)
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

cur = conn.cursor() # create cursor for our opened connection in object conn

query_to_run = """
    SELECT *
    FROM eda.king_county_house_details
    JOIN eda.king_county_house_sales
    ON eda.king_county_house_details.id = eda.king_county_house_sales.house_id;
"""


cur.fetchall() # gets data called by query
df_psycopg = pd.read_sql(query_to_run, conn) # read queried data from SQL database into pandas dataframe

conn.close() #close the connection


In [ ]:
#### same with  SQLALCHEMY
from sqlalchemy import create_engine # for creating an engine

#read the database string DB_STRING from the .env
load_dotenv()

DB_STRING = os.getenv('DB_STRING') # gets database string DB_STRING from .env file and assigns it as value for new variable DB_STRING
db = create_engine(DB_STRING) # creates engine from database string DB_STRING
#import the data to a pandas dataframe
query_string = "SELECT * FROM datasets.kaggle_survey" # write SQL-query into variable query_string
df_sqlalchemy = pd.read_sql(query_string, db) # read queried data from SQL database into pandas dataframe


### SQL & Python shorts: 

Python: 

To just execute an SQL query without download 

>```cur.execute(query_to_run) ```

SQL: 

## Read and write CSV files 

In [ ]:
#export the data to a csv-file
df_sqlalchemy.to_csv('kaggle_survey.csv',index=False)

# read csv table into the dataframe
df_joined_table = pd.read_csv('kaggle_survey.csv') 

# Working with Data
#### Creating data frames with Pandas  

In [ ]:
# Imports
import numpy as np
import pandas as pd


In [ ]:
# 0: from a CSV 
df0 = pd.read_csv('kaggle_survey.csv') 

# 1: Form a list of dictionary 
data_lst = [{'a': 1}, {'b':5}, {'c': 4}] # col will be a, b, c 
df1 = pd.DataFrame(data_lst)

# 2: From a Numpy array 
data2 = np.array([[1, 2, 3], [4, 5, 6]]) # each index will be 
df2 = pd.DataFrame(data2, columns=['Column1', 'Column2', 'Column3']) 

# 3: From a dictionary 
data3 = {
    'Column1': [1, 2, 3, 4],
    'Column2': ['A', 'B', 'C', 'D']
}

df3 = pd.DataFrame(data3)


#### Getting an insight of the data   

In [ ]:
df0.head() # show top of the data frame, can take an integer in () to limit the amount of lines 

df0.tail() # same with bottom 

df0.shape # will return us the number of rows and columns of our DataFrame

df1.columns # will show col names 

df3.info() # will output Non-Null Count and data type for each col 

df2.describe() # to get some statistics, min, max, mean, quartiles, std for numeric values 

df4 = df2.copy() # make a copy of your dataframe



#### Manipulate the column names and stuff


In [ ]:
# make a list column names
cols = df2.columns.tolist()

# replace space with _
cols = [col.replace(' ', '_') for col in cols]
# reassign new column names to dataframe
df2.columns = cols

#### Work with data in the data frame


In [ ]:

df3['Column1'] # grab a column in bracket notation 

df3.Column1 # grab a column in dot notation 

df3[['Column1', 'Column2']] # can also grab more than one col 

df3[:3] # This will grab from the beginning up to but not including the row at index 3. So rows 0, 1, 2 


#### Work with data in the data frame
